In [1]:
import re
import os
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [6]:
def clean_replics(repl):
    filt_s = re.compile('\n|\t')
    filt_tags = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    filt_spl = re.compile('\s\s+')
    filt_emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    filt_url = re.compile(r'https?:\/\/.*[\r\n]*')
    filt_names = re.compile('@\w+')
  
    clean_repl = filt_tags.sub(' ', repl)
    clean_repl = filt_s.sub(' ', clean_repl)
    clean_repl = filt_spl.sub(' ', clean_repl)
    clean_repl = filt_emoji.sub(' ', clean_repl)
    clean_repl = filt_url.sub(' ', clean_repl)
    clean_repl = filt_names.sub(' ', clean_repl)
    return clean_repl.replace('  ', ' ').strip()

In [7]:
def clean_from_rubbish_dota(repl):
    authers_param = re.compile(r'.* [(написал)|(said)].*:')
    cit_param = re.compile(r'(Цитата)')
    like_param = re.compile(r'(.*\n*.*)?(понравилось это)', re.DOTALL)
    find_ins = re.findall(r'\n*\t*\n*\t*(.*)', repl)

    find_extra_likes = re.compile(r'([A-Za-z]+[\,и]?[A-Za-z]+)')
  # print(find_ins)
    list_replics = [el for el in find_ins if ('Показать больше' not in el and el != '' and
                  'Изменено' not in el and 'Показать содержимое' not in el and
                  re.search(authers_param, el) is None and
                  re.search(cit_param, el) is None and 
                  re.search(like_param, el) is None and
                   re.search(find_extra_likes, el) is None)]
  
    new_str = ' '.join(list_replics)
    return new_str

In [8]:
def make_dataset(data):
    repl1_list = []
    repl2_list = []

    for ind, row in tqdm(data.iterrows()):
      
        if_extra = []
      # print(isinstance(row[1], str))
      # if type()
        if isinstance(row[0], str) is True:
            replic1 = clean_from_rubbish_dota(str(row[0]))
            repl1_list.append(clean_replics(str(replic1)))
        else:
        # replic1 = np.nan
            repl1_list.append(np.nan)
        if isinstance(row[1], str) is True:
            replic2 = clean_from_rubbish_dota(str(row[1]))
            repl2_list.append(clean_replics(str(replic2)))
        else:
            replic2 = np.nan
            repl2_list.append(np.nan)
    dota_df = pd.DataFrame({'Реплика 1': repl1_list, 'Реплика 2': repl2_list})
    return dota_df

In [9]:
# dota_df = pd.DataFrame({'Реплика 1': repl1_list, 'Реплика 2': repl2_list})

In [10]:
# dota_df

In [11]:
def cleaning_dataset(df):
    list_full = []

    reps1 = []
    reps2 = []

    df.drop_duplicates(subset=['Реплика 1', 'Реплика 2'], keep='first')

    for ind, row in df.iterrows():
        if row[0] not in list_full and row[1] not in list_full:
            list_full.append(row[0])
            reps1.append(row[0])
            list_full.append(row[1])
            reps2.append(row[1])
  
    final_df = pd.DataFrame({'Реплика 1': reps1, 'Реплика 2': reps2})

    final_df['Реплика 2'].replace('', np.nan, inplace=True)
    final_df['Реплика 2'].replace(' ', np.nan, inplace=True)
    final_df['Реплика 1'].replace('', np.nan, inplace=True)
    final_df['Реплика 1'].replace(' ', np.nan, inplace=True)

    dota = final_df.dropna(axis=0)
    return dota

In [12]:
# dota_all = cleaning_dataset(dota_df)

In [13]:
# dota_all

In [14]:
# dota_all.to_csv('dota_dataset.csv', index=False)

In [17]:
# итерируемся по всем файлам с продоты
for dirpath, dirnames, filenames in os.walk("./dota_files"):
    # print(filenames)
    for file in filenames:
        filename = dirpath + "/" + file
        # print(filename)
        print(file)
        fl = str(file).split('.')
        print(fl[0])
        cur_df = pd.read_csv(filename, header=0,  names=['Реплика 1', 'Рeплика 2'])
        print(cur_df.shape)
        # data = data.append(cur_df, ignore_index=True)
        dota_df = make_dataset(cur_df)
        dota_all = cleaning_dataset(dota_df)
        dota_all.to_csv(f'dataset_{fl[0]}.csv', index=False)

dota217336.csv
dota217336
(8358, 2)



dota210667.csv
dota210667
(620, 2)



dota218207.csv
dota218207
(11269, 2)



dota156892.csv
dota156892
(349, 2)



dota218775.csv
dota218775
(12960, 2)



dota218957.csv
dota218957
(8379, 2)



dota220172.csv
dota220172
(11302, 2)



dota213960.csv
dota213960
(238, 2)



dota220574.csv
dota220574
(5024, 2)


KeyboardInterrupt: 